# Adding context to your prompts - Retrieval Augmented Generation (RAG)

## Install OpenAI .NET SDK & other NuGet packages

In [9]:
#r "nuget: Azure.AI.OpenAI, 1.0.0-beta.15"
#r "nuget: Microsoft.DotNet.Interactive, 1.0.0-beta.24164.1"
#r "nuget: System.Numerics.Tensors, 8.0.0"

Installed Packages Azure.AI.OpenAI, 1.0.0-beta.15 Microsoft.DotNet.Interactive, 1.0.0-beta.24164.1 System.Numerics.Tensors, 8.0.0

In [10]:
using System.Collections.Generic;
using System.IO;
using System.Linq;
using System.Text.Json;
using Azure.AI.OpenAI;
using Microsoft.DotNet.Interactive;
using System.Numerics.Tensors;

## Get Azure OpenAI Service credentials

In [11]:
var endpoint = Environment.GetEnvironmentVariable("AOAI_ENDPOINT");
var key = Environment.GetEnvironmentVariable("AOAI_KEY");
var deploymentId = Environment.GetEnvironmentVariable("AOAI_DEPLOYMENTID");
var embeddingDeploymentId = Environment.GetEnvironmentVariable("AOAI_EMBEDDINGS_DEPLOYMENTID");

## Configure OpenAI client

In [12]:
var client = new OpenAIClient(new Uri(endpoint), new Azure.AzureKeyCredential(key));

## Define system prompt

In [13]:
var systemPrompt = 
    """
    You are an AI Assistant that helps user answer questions about their data. 
    
    You have access to articles to help you answer questions. 
    
    Use only the contents from these documents to answer questions. 
    """;

## Define Data Objects

In [14]:
class Metadata
{
    public string Title {get;set;}
    public string Text  {get;set;}
    public long WikiId  {get;set;}
    public int ParagraphId  {get;set;}
}


class Document
{
    public float[] Embedding {get;set;}
    public Metadata Metadata {get;set;}
}

## Utility Functions

In [15]:
string FormatMessage(ChatRequestMessage m)
{
    if(m.Role == ChatRole.System)
        return $"System: {((ChatRequestSystemMessage)m).Content }";
    else if(m.Role == ChatRole.User)
        return $"User: {((ChatRequestUserMessage)m).Content}";
    else if(m.Role == ChatRole.Assistant)
        return $"Assistant: {((ChatRequestAssistantMessage)m).Content}";
    else
        return m.ToString();
}

var RenderUserPrompt = (List<ChatRequestMessage> history, string[] documents, string query) => 
    {
        var chatMessages = 
            history
                .Select(FormatMessage)
                .Select(x => $"{x}\n")
                .ToArray();

        var sources = String.Join('\n', documents);

        var template = 
            $"""
            ## Chat History

            {String.Join('\n',chatMessages)}

            ## Sources

            {sources}

            ## Query
            {query}
            """;

        return template;
    };

var GetTopKDocuments = (Document[] sources, float[] userQueryEmbedding, int k) => 
{
    return sources
        .Select(document => 
            {
                var similarity = TensorPrimitives.CosineSimilarity(document.Embedding, userQueryEmbedding);
                return (similarity, document);
            })
        .OrderByDescending(x => x.Item1)
        .Take(k)
        .Select(x => x.Item2)
        .ToArray();
};

## Load data

In [16]:
var data = File.ReadAllText("../Data/embeddings.json");
var sources = JsonSerializer.Deserialize<Document[]>(data);

## Display first data entry

In [17]:
sources.First()

Submission#13+Document Embedding [ 0.015050261, 0.009753172, 0.010801293, -0.031380862, -0.0005856453, -0.010518865, -0.004339973, -0.015000052, -0.016644409, 0.0016459262, 0.026535656, 0.014121388, -0.03469468, -0.01562767, -0.046393465, 0.0054037846, 0.018401738, 0.008039777, 0.007945634, -0.023573304 ... (1516 more) ] Metadata Submission#13+Metadata Title 24-hour clock Text The 24-hour clock is a way of telling the time in which the day runs from midnight to midnight and is divided into 24 hours, numbered from 0 to 23. It does not use a.m. or p.m. This system is also referred to (only in the US and the English speaking parts of Canada) as military time or (only in the United Kingdom and now very rarely) as continental time. In some parts of the world, it is called railway time. Also, the international standard notation of time (ISO 8601) is based on this format. WikiId 9985 ParagraphId 0

## Configure Chat Completion options

In [18]:
var options = new ChatCompletionsOptions
{
    DeploymentName=deploymentId,
    MaxTokens=400,
    Temperature=1f,
    FrequencyPenalty=0.0f,
    PresencePenalty=0.0f,
    NucleusSamplingFactor = 0.95f // Top P
};

## Initialize chat history

In [19]:
var messages = new List<ChatRequestMessage>();
var systemMessage = new ChatRequestSystemMessage(systemPrompt);
messages.Add(systemMessage);
options.Messages.Add(systemMessage);

## Start chat

In [20]:
var chatting = true;

In [21]:
Console.WriteLine($"{FormatMessage(systemMessage)}");

while(chatting)
{
    //Get User input
    var userInput = await Kernel.GetInputAsync("Please enter your prompt. Press 'q' to quit.");
    if(userInput.ToLowerInvariant() == "q")
    {
        chatting = false;
        break;    
    }
    
    // Query relevant documents
    var embeddingOptions = new EmbeddingsOptions
    {
        DeploymentName=embeddingDeploymentId,
        Input={userInput}
    };

    var embeddingResponse = await client.GetEmbeddingsAsync(embeddingOptions);
    var inputEmbedding = embeddingResponse.Value.Data[0].Embedding.ToArray();
    var docs = 
        GetTopKDocuments(sources, inputEmbedding, 3)
            .Select(doc => doc.Metadata.Text)
            .ToArray(); 

    // Generate User Prompt
    messages.Add(new ChatRequestUserMessage(userInput));
    var query = RenderUserPrompt(messages, docs, userInput);
    var userMessage = new ChatRequestUserMessage(query);
    options.Messages.Add(userMessage);
    Console.WriteLine(query);
    
    // Generate response
    var assistantResponse = await client.GetChatCompletionsAsync(options);
    var response = assistantResponse.Value.Choices[0].Message.Content;
    var assistantMessage = new ChatRequestAssistantMessage(response);
    Console.WriteLine($"\n## Response\n{FormatMessage(assistantMessage)}\n");
    messages.Add(assistantMessage);
    options.Messages.Add(assistantMessage);
    Console.WriteLine("-----");
    Console.WriteLine("");
};

messages.Clear();
options.Messages.Clear();

System: You are an AI Assistant that helps user answer questions about their data. 

You have access to articles to help you answer questions. 

Use only the contents from these documents to answer questions. 
## Chat History

System: You are an AI Assistant that helps user answer questions about their data. 

You have access to articles to help you answer questions. 

Use only the contents from these documents to answer questions. 

User: How many goals did Cristiano Ronaldo score in 2014?


## Sources

In the 2013-14 season, Ronaldo broke the record for most goals in one Champions League season by scoring his 17th goal with a penalty in extra time in the final against Atlético Madrid that Real Madrid won 4–1. The previous record was 14 goals, set by Messi in the 2011-12 season.
In the 2014-15 season, Ronaldo set a new personal record by scoring 61 goals in all competitions. This achievement helped him win his second Ballon d'Or. He scored five goals in one match for the first time in